In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from model.ewc_regression import ElasticWeightConsolidation
import numpy as np

/home/po/miniconda3/envs/py36/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
class CustomDataset(Dataset):
    def __init__(self):
        x = np.linspace(1,10,200, dtype="float64") 
        y = x**2 + 1
        x = x.reshape(len(x),1)
        y = y.reshape(len(y),1)
        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [3]:
mnist_train = CustomDataset()
mnist_test = CustomDataset()
train_loader = DataLoader(mnist_train, batch_size = 200, shuffle=False)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

In [4]:
class LinearLayer(nn.Module):
    def __init__(self, input_dim, output_dim, act='relu', use_bn=False):
        super(LinearLayer, self).__init__()
        self.use_bn = use_bn
        self.lin = nn.Linear(input_dim, output_dim)
        self.act = nn.ReLU() if act == 'relu' else act
        if use_bn:
            self.bn = nn.BatchNorm1d(output_dim)
    def forward(self, x):
        if self.use_bn:
            return self.bn(self.act(self.lin(x)))
        return self.act(self.lin(x))

class Flatten(nn.Module):

    def forward(self, x):
        return x.view(x.shape[0], -1)


In [5]:
class BaseModel(nn.Module):
    
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(BaseModel, self).__init__()
        self.f1 = Flatten()
        self.lin1 = LinearLayer(num_inputs, num_hidden, use_bn=False)
        self.lin2 = LinearLayer(num_hidden, num_hidden, use_bn=False)
        self.lin3 = nn.Linear(num_hidden, num_outputs)
        
    def forward(self, x):
        return self.lin3(self.lin2(self.lin1(self.f1(x))))

In [6]:
crit = nn.MSELoss(reduce="sum")
ewc = ElasticWeightConsolidation(BaseModel(1, 200, 1), crit=crit, lr=1e-3)
ewc.model.double()

/home/po/miniconda3/envs/py36/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


BaseModel(
  (f1): Flatten()
  (lin1): LinearLayer(
    (lin): Linear(in_features=1, out_features=200, bias=True)
    (act): ReLU()
  )
  (lin2): LinearLayer(
    (lin): Linear(in_features=200, out_features=200, bias=True)
    (act): ReLU()
  )
  (lin3): Linear(in_features=200, out_features=1, bias=True)
)

In [7]:
from tqdm import tqdm

In [8]:
STEPS = 4000
# scheduler = torch.optim.lr_scheduler.MultiStepLR(ewc.optimizer, milestones=[0.5 * STEPS, 0.75 * STEPS], gamma=0.1)
for _ in range(STEPS):
    for input, target in train_loader:
        ewc.forward_backward_update(input, target)
#         scheduler.step()
        

loss: tensor(2233.1077, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(2124.7996, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(2022.6135, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1924.2236, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1832.6403, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1744.5945, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1657.2904, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1569.9524, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1482.3481, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1394.2100, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1304.8625, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1215.1357, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1125.4539, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(1037.1802, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(950.0334, dtype=torch

loss: tensor(51.1243, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(49.4766, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(47.8640, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(46.2903, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(44.7585, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(43.2764, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(41.8501, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(40.4821, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(39.1774, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(37.9393, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(36.7622, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(35.6299, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(34.5396, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(33.4917, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(32.4859, dtype=torch.float64, grad_fn=<AddBackwar

loss: tensor(0.3713, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3622, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3536, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3454, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3375, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3300, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3229, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3163, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3105, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3059, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3034, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3038, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3081, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3135, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.3132, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

loss: tensor(0.0480, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0476, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0473, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0469, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0466, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0463, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0461, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0459, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0457, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0458, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0460, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0466, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0477, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0495, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0525, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

loss: tensor(0.0228, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0181, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0256, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0314, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0260, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0184, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0188, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0242, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0251, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0203, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0170, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0191, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0219, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0209, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0176, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

...

loss: tensor(0.0006, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0008, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0008, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0006, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0006, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0008, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0011, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

loss: tensor(0.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

loss: tensor(0.0384, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0248, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0079, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0046, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0141, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0202, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0176, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0097, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0024, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0035, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0081, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0101, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tensor(0.0078, dtype=torch.float64, grad_fn=<AddBackward0>)
loss: tens

In [9]:
model = ewc.model.eval()
input = torch.tensor([3],dtype=torch.float64)
print(model(input))
input = torch.tensor([25],dtype=torch.float64)
print(25**2 +1)
print(model(input))

ewc.model.train()

tensor([[10.0026]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
626
tensor([[368.3151]], dtype=torch.float64, grad_fn=<AddmmBackward0>)


BaseModel(
  (f1): Flatten()
  (lin1): LinearLayer(
    (lin): Linear(in_features=1, out_features=200, bias=True)
    (act): ReLU()
  )
  (lin2): LinearLayer(
    (lin): Linear(in_features=200, out_features=200, bias=True)
    (act): ReLU()
  )
  (lin3): Linear(in_features=200, out_features=1, bias=True)
)

In [10]:
ewc.register_ewc_params(mnist_train, 200, 1)

ewc.model.eval()
input = torch.tensor([3],dtype=torch.float64)
print(model(input))
input = torch.tensor([25],dtype=torch.float64)
print(25**2 +1)
print(model(input))

ewc.model.train()

tensor([[10.0026]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
626
tensor([[368.3151]], dtype=torch.float64, grad_fn=<AddmmBackward0>)


BaseModel(
  (f1): Flatten()
  (lin1): LinearLayer(
    (lin): Linear(in_features=1, out_features=200, bias=True)
    (act): ReLU()
  )
  (lin2): LinearLayer(
    (lin): Linear(in_features=200, out_features=200, bias=True)
    (act): ReLU()
  )
  (lin3): Linear(in_features=200, out_features=1, bias=True)
)

In [11]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
class CustomDataset(Dataset):
    def __init__(self):
        x = np.linspace(20,30,200, dtype="float64") 
        y = x**2 + 1
        x = x.reshape(len(x),1)
        y = y.reshape(len(y),1)
        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y
mnist_train = CustomDataset()
mnist_test = CustomDataset()
train_loader = DataLoader(mnist_train, batch_size = 200, shuffle=False)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

In [11]:
for _ in range(4000):
    for input, target in tqdm(train_loader):
        ewc.forward_backward_update(input, target)

100%|██████████| 1/1 [00:00<00:00, 10.23it/s]


loss: tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)


100%|██████████| 1/1 [00:00<00:00, 10.68it/s]


loss: tensor(0.0048, dtype=torch.float64, grad_fn=<AddBackward0>)


100%|██████████| 1/1 [00:00<00:00, 24.51it/s]


...

loss: tensor(2.6339, dtype=torch.float64, grad_fn=<AddBackward0>)


100%|██████████| 1/1 [00:00<00:00, 120.70it/s]


loss: tensor(2.4529, dtype=torch.float64, grad_fn=<AddBackward0>)


100%|██████████| 1/1 [00:00<00:00, 122.26it/s]

loss: tensor(2.3499, dtype=torch.float64, grad_fn=<AddBackward0>)


In [12]:
model = ewc.model.eval()
input = torch.tensor([3],dtype=torch.float64)
print(model(input))
input = torch.tensor([25],dtype=torch.float64)
print(model(input))
print("after==============")

ewc.model.train()
ewc.register_ewc_params(mnist_train, 200, 1)
model = ewc.model.eval()
input = torch.tensor([3],dtype=torch.float64)
print(model(input))
input = torch.tensor([25],dtype=torch.float64)
print(model(input))

tensor([[9.9964]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
tensor([[368.2339]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
after==============
tensor([[9.9964]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
tensor([[368.2339]], dtype=torch.float64, grad_fn=<AddmmBackward0>)


In [32]:
def accu(model, dataloader):
    model = model.eval()
    acc = 0
    for input, target in dataloader:
        o = model(input)
        acc += (o.argmax(dim=1).long() == target).float().mean()
    return acc / len(dataloader)

In [33]:
accu(ewc.model, f_test_loader)

tensor(0.8129)

In [34]:
accu(ewc.model, test_loader)

tensor(0.6420)